In [1]:
#NOTE: use paimg1 env, the retccl one has package issue with torchvision
import sys
import os
import numpy as np
import openslide
import matplotlib.pyplot as plt

import matplotlib
matplotlib.use('Agg')
import pandas as pd
import warnings
import torch
import torch.nn as nn

from sklearn.model_selection import KFold, train_test_split
from torch.utils.data import DataLoader
import torch.optim as optim
from pathlib import Path
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

sys.path.insert(0, '../Utils/')
from Utils import create_dir_if_not_exists
from Utils import generate_deepzoom_tiles, extract_tile_start_end_coords, get_map_startend
from Utils import get_downsample_factor
from Utils import minmax_normalize, count_label
from Utils import log_message, set_seed
from Utils import simple_line_plot
from cluster_utils import get_cluster_data, get_cluster_label, get_updated_feature, get_pca_components
from Eval import compute_performance, plot_LOSS, compute_performance_each_label, get_attention_and_tileinfo
from train_utils import pull_tiles, get_feature_label_array_dynamic
from train_utils import ModelReadyData_diffdim, convert_to_dict, prediction
from Model import Mutation_MIL_MT
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
####################################
######      USERINPUT       ########
####################################
SELECTED_LABEL = ["AR","MMR (MSH2, MSH6, PMS2, MLH1, MSH3, MLH3, EPCAM)2","PTEN","RB1","TP53","TMB_HIGHorINTERMEDITATE","MSI_POS"]
SELECTED_FEATURE = [str(i) for i in range(0,2048)] + ['TUMOR_PIXEL_PERC']
TUMOR_FRAC_THRES = 0.9
TRAIN_SAMPLE_SIZE = "ALLTUMORTILES"
TRAIN_OVERLAP = 100
TEST_OVERLAP = 0
SELECTED_FOLD = 0
CLUSTER_ALG = 'KMEAN'
N_CLUSTERS = 3
CLUSTER_DIST = 'L2'
feature_extraction_method = 'retccl'

##################
###### DIR  ######
##################
proj_dir = '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/'
data_dir = proj_dir + 'intermediate_data/model_ready_data/feature_' + feature_extraction_method + '/MAXSS'+ str(TRAIN_SAMPLE_SIZE)  + '_TrainOL' + str(TRAIN_OVERLAP) +  '_TestOL' + str(TEST_OVERLAP) + '_TFT' + str(TUMOR_FRAC_THRES) + "/split_fold" + str(SELECTED_FOLD) + "/"
data_info_path =  data_dir + "init_data_info/" 
save_name = "_NCLUSTER_" + str(N_CLUSTERS) +  "_DISTMETRIC_" + CLUSTER_DIST

################################################
#Create output dir
################################################
outdir =   os.path.join(data_dir, "clusters", CLUSTER_ALG)
create_dir_if_not_exists(outdir)

outdir1 =   os.path.join(outdir, "DL_Updated_Features_EmbeddingWithClusterNumber")
create_dir_if_not_exists(outdir1)

outdir2 =   os.path.join(outdir, "ClusterInfo")
create_dir_if_not_exists(outdir2)

                         
outdir3 =   os.path.join(outdir, "ML_Updated_Features_OnlyClusterPerc")
create_dir_if_not_exists(outdir3)

##################
#Select GPU
##################
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
set_seed(0)

Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/model_ready_data/feature_retccl/MAXSSALLTUMORTILES_TrainOL100_TestOL0_TFT0.9/split_fold0/clusters/KMEAN' already exists.
Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/model_ready_data/feature_retccl/MAXSSALLTUMORTILES_TrainOL100_TestOL0_TFT0.9/split_fold0/clusters/KMEAN/DL_Updated_Features_EmbeddingWithClusterNumber' already exists.
Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/model_ready_data/feature_retccl/MAXSSALLTUMORTILES_TrainOL100_TestOL0_TFT0.9/split_fold0/clusters/KMEAN/ClusterInfo' already exists.
Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/model_ready_data/feature_retccl/MAXSSALLTUMORTILES_TrainOL100_TestOL0_TFT0.9/split_fold0/clusters/KMEAN/ML_Updated_Features_OnlyClusterPerc' already exists.
cuda:0


In [3]:
############################################################################################################
#Get features and labels
############################################################################################################
train_feature = torch.load(data_info_path + 'train_feature.pth')
test_feature = torch.load(data_info_path + 'test_feature.pth')
val_feature = torch.load(data_info_path + 'val_feature.pth')

train_label = torch.load(data_info_path + 'train_label.pth')
test_label = torch.load(data_info_path + 'test_label.pth')
val_label = torch.load(data_info_path + 'val_label.pth')

train_ids = torch.load(data_info_path + 'train_ids.pth')
test_ids = torch.load(data_info_path + 'test_ids.pth')
val_ids = torch.load(data_info_path + 'val_ids.pth')


train_info = torch.load(data_info_path + 'train_info.pth')
test_info = torch.load(data_info_path + 'test_info.pth')
val_info = torch.load(data_info_path + 'val_info.pth')
train_info_df = pd.concat(train_info)
test_info_df = pd.concat(test_info)
val_info_df = pd.concat(val_info)

#Combine feature and label 
train_feature_df = get_cluster_data(train_feature, train_label, train_ids, SELECTED_LABEL)
test_feature_df = get_cluster_data(test_feature, test_label, test_ids, SELECTED_LABEL)
valid_feature_df = get_cluster_data(val_feature, val_label, val_ids, SELECTED_LABEL)


KeyboardInterrupt



In [ ]:
################################################
#Clustering
################################################
cluster_feature_name = range(0,2048)
cluster_feature = train_feature_df[cluster_feature_name]

# # Calculate WCSS for different values of k
# wcss = []
# for i in range(1, 11):
#     print(i)
#     kmeans = KMeans(n_clusters=i, random_state=42)
#     kmeans.fit(cluster_feature)
#     wcss.append(kmeans.inertia_)

# # Plot the Elbow graph
# simple_line_plot(range(1, 11), wcss, 'Number of clusters (k)', 'WCSS', 'Elbow Method', outdir2 + "/Elbow_Ks.png")

In [ ]:
# Perform K-means clustering using N_CLUSTERS K
kmeans = KMeans(n_clusters= N_CLUSTERS)
kmeans.fit(cluster_feature)

# Get cluster centers and labels
centers = kmeans.cluster_centers_
cluster_labels_train = kmeans.labels_

#store center df
center_list = [pd.DataFrame(center).T for center in centers]
center_df = pd.concat(center_list)

In [ ]:
#Get Cluster labels for test and val
cluster_labels_test = get_cluster_label(test_feature_df, centers, cluster_feature_name)
cluster_labels_val = get_cluster_label(valid_feature_df, centers, cluster_feature_name)

#add cluster label to df
train_feature_df['Cluster'] = cluster_labels_train
test_feature_df['Cluster'] = cluster_labels_test
valid_feature_df['Cluster'] = cluster_labels_val

In [ ]:
pcs_train = get_pca_components(train_feature_df[cluster_feature_name], n_components = 2)
pcs_test = get_pca_components(test_feature_df[cluster_feature_name], n_components = 2)
pcs_val = get_pca_components(valid_feature_df[cluster_feature_name], n_components = 2)
pcs_center = get_pca_components(center_df[cluster_feature_name], n_components = 2)

In [ ]:
#Add cluster label to info df
train_info_df['Cluster'] = cluster_labels_train
test_info_df['Cluster'] = cluster_labels_test
val_info_df['Cluster'] = cluster_labels_val
train_info_df['PC1'] = pcs_train[:, 0]
train_info_df['PC2'] = pcs_train[:, 1]
test_info_df['PC1'] = pcs_test[:, 0]
test_info_df['PC2'] = pcs_test[:, 1]
val_info_df['PC1'] = pcs_val[:, 0]
val_info_df['PC2'] = pcs_val[:, 1]
train_info_df[SELECTED_LABEL] = train_feature_df[SELECTED_LABEL]
test_info_df[SELECTED_LABEL] = test_feature_df[SELECTED_LABEL]
val_info_df[SELECTED_LABEL] = valid_feature_df[SELECTED_LABEL]

train_info_df.to_csv(outdir2 + '/train_cluster_info' + save_name + '.csv', index= False)
test_info_df.to_csv(outdir2 + '/test_cluster_info' + save_name + '.csv',index= False)
val_info_df.to_csv(outdir2 + '/valid_cluster_info' + save_name + '.csv',index= False)

In [ ]:
#Get closest to center pcs
distances = cdist(train_feature_df[cluster_feature_name], center_df[cluster_feature_name].to_numpy(), 'euclidean')
closest_indices = np.argmin(distances, axis=0)
cloest_tocenter_pcs = pcs_train[closest_indices]

In [ ]:
# Plot the data points and cluster centers with cluster labels
plt.figure(figsize=(10, 6))
plt.scatter(pcs_train[:, 0], pcs_train[:, 1], c=cluster_labels_train, cmap='viridis',alpha=0.6, s = 0.01)
for i, center in enumerate(cloest_tocenter_pcs):
    plt.scatter(center[0], center[1], c='red', marker=f'${i}$', s=200)  # Use cluster label as marker

plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Clustering for Tile Embeddings')
plt.grid(True)
plt.savefig(outdir2  + '/original_cluster_scatter' + save_name + '.png')
plt.close()

In [ ]:
#Get updated feature for deep learning
updated_feature_names= list(range(0,2048)) + ['TUMOR_PIXEL_PERC','Cluster']
updated_train_feature = get_updated_feature(train_feature_df, train_ids, updated_feature_names)
updated_test_feature = get_updated_feature(test_feature_df, test_ids, updated_feature_names)
updated_val_feature = get_updated_feature(valid_feature_df, val_ids, updated_feature_names)

torch.save(updated_train_feature, outdir1 + '/updated_train_feature' + save_name + '.pth')
torch.save(updated_test_feature,  outdir1 + '/updated_test_feature'  + save_name + '.pth')
torch.save(updated_val_feature,   outdir1 + '/updated_val_feature' + save_name + '.pth')

In [ ]:
#Get feature for machine learning
#For each sample, compute cluster percentage

def get_percentage_clusters(feature_df):
    grouped = feature_df.groupby('ID')['Cluster'].value_counts(normalize=True).reset_index(name='Percentage')
    cluster_perc_df = pd.DataFrame(grouped.pivot_table(index='ID', columns='Cluster', values='Percentage', fill_value=0))
    cluster_perc_df = cluster_perc_df.add_prefix('C_')
    cluster_perc_df = cluster_perc_df.reset_index(drop = False)

    return cluster_perc_df

train_cluster_perc_df = get_percentage_clusters(train_feature_df)
test_cluster_perc_df = get_percentage_clusters(test_feature_df)
valid_cluster_prec_df = get_percentage_clusters(valid_feature_df)

In [ ]:
torch.save(train_cluster_perc_df, outdir3 + '/updated_train_feature' + save_name + 'cluster_perc.pth')
torch.save(test_cluster_perc_df,  outdir3 + '/updated_test_feature'  + save_name + 'cluster_perc.pth')
torch.save(valid_cluster_prec_df, outdir3 + '/updated_val_feature' + save_name + 'cluster_perc.pth')